### 0. Import Libraries

In [1]:
import torch
import torchvision

### 1.1 Hyperparameters

In [2]:
inputs=784 + 1 # bias input
hidden=100
outputs=10
ticks = 3

batch_size=64
epochs = 100
lr = 0.001

### 1.2 Dataset

In [3]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST("/tmp/mnist_data", train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST("/tmp/mnist_data", train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size, shuffle=True)

### 2 MNN forward definition

In [4]:
def net(s, x):
    s[:,0:inputs-1] = x
    s[:,inputs] = 1
    t = torch.matmul(s, A)
    s = torch.relu(t)
    return s

### 3. Training Loop

In [ ]:
neurons = inputs + hidden + outputs
A = torch.rand(neurons, neurons).requires_grad_(True)
optimizer = torch.optim.Adam([A], lr=lr)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(0, epochs):
    for i, (batch_X, batch_y) in enumerate(train_loader):
        state = torch.zeros(batch_X.shape[0], neurons)
        for t in range(0, ticks):
            state = net(state, batch_X.view(-1, 28*28))
            
        outs = state[:,inputs+hidden:neurons]
        loss = loss_fn(outs, batch_y)
        
        # Using pytorch backpropagation becouse our numpy implementaiton is inefficent
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc = (outs.max(1).indices == batch_y).float().mean()
        
        #mean_loss = 0.01*loss.item() + 0.99*mean_loss
        #mean_acc = 0.01*acc.item() + 0.99*mean_acc
        
        if i % 200 == 0:
            print("epoch: %d  loss: %f  acc: %f" % (epoch, loss.item(), acc.item()))


epoch: 0  loss: 3372.194092  acc: 0.062500
epoch: 0  loss: 26.474407  acc: 0.359375
epoch: 0  loss: 6.621606  acc: 0.421875
epoch: 0  loss: 8.578157  acc: 0.515625
epoch: 0  loss: 5.717260  acc: 0.390625
epoch: 1  loss: 27.072432  acc: 0.468750
epoch: 1  loss: 8.286741  acc: 0.375000
epoch: 1  loss: 2.454543  acc: 0.468750
epoch: 1  loss: 6.169957  acc: 0.500000
epoch: 1  loss: 1.970924  acc: 0.390625
epoch: 2  loss: 7.954168  acc: 0.421875
epoch: 2  loss: 1.510525  acc: 0.375000
epoch: 2  loss: 30.964554  acc: 0.281250
epoch: 2  loss: 6.005965  acc: 0.343750
epoch: 2  loss: 6.202227  acc: 0.312500
epoch: 3  loss: 25.473427  acc: 0.359375
epoch: 3  loss: 1.553840  acc: 0.390625
epoch: 3  loss: 3.897541  acc: 0.296875
epoch: 3  loss: 10.986524  acc: 0.328125
epoch: 3  loss: 1.484676  acc: 0.375000
epoch: 4  loss: 4.381802  acc: 0.359375
epoch: 4  loss: 12.233021  acc: 0.312500
epoch: 4  loss: 8.849322  acc: 0.203125
epoch: 4  loss: 1.799240  acc: 0.234375
epoch: 4  loss: 19.747335  acc:

### 4. Testing Loop

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

y_true = []
y_pred = []

for i, (batch_X, batch_y) in enumerate(test_loader):
    state = torch.zeros(batch_X.shape[0], neurons)
    for t in range(0, ticks):
        state = net(state, batch_X.view(-1, 28*28))

    outs = state[:,inputs+hidden:neurons]
    preds = outs.max(1).indices
    
    y_true.extend(batch_y.detach().numpy().tolist())
    y_pred.extend(preds.detach().numpy().tolist())

print("Accuracy:")
print(accuracy_score(y_true, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))